In [1]:
import os, sys, json, scipy
import numpy as np

In [2]:
mat = np.array([[2, 6], [1, 3], [3, 9], [5, 15]])
mat

array([[ 2,  6],
       [ 1,  3],
       [ 3,  9],
       [ 5, 15]])

In [3]:
eigenval, eigenvec = np.linalg.eigh(np.matmul(mat.transpose(), mat))
eigenvec = eigenvec[np.argsort(eigenval)[::-1]].transpose()
eigenval = np.sort(eigenval)[::-1]
eigenval

array([390.,   0.])

In [4]:
eigenvec

array([[ 0.31622777, -0.9486833 ],
       [ 0.9486833 ,  0.31622777]])

In [5]:
svd_left = np.matmul(mat, eigenvec)
svd_left

array([[ 6.32455532e+00, -2.22044605e-16],
       [ 3.16227766e+00, -1.11022302e-16],
       [ 9.48683298e+00, -1.11022302e-16],
       [ 1.58113883e+01, -7.77156117e-16]])

In [6]:
mat_shape = mat.shape

S = np.zeros(mat_shape)
for i in range(len(eigenval)):
    S[i][i] = np.sqrt(eigenval[i])

S

array([[19.74841766,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [7]:
UT = np.zeros((mat_shape[0], mat_shape[0]))
UT

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
for i in range(len(eigenval)):
    if eigenval[i] > 0:
        u = svd_left.transpose()[i] / S[i][i]
        UT[i] = u

In [9]:
def GenOrthVec(UT: np.array):
    mat_shape = UT.shape[0]
    rawgram = np.random.dirichlet(tuple([1 for i in range(mat_shape)]))
    rawgram /= scipy.linalg.norm(rawgram)
    gram = rawgram - np.dot(rawgram, UT[0]) * UT[0]

    for i in range(1, mat_shape):
        gram -= np.dot(gram, UT[i]) * UT[i]
    gram /= scipy.linalg.norm(gram)
    
    return gram

In [10]:
for i in range(mat_shape[0]):
    if scipy.linalg.norm(UT[i]) == 0.0:
        UT[i] = GenOrthVec(UT)

UT

array([[ 0.32025631,  0.16012815,  0.48038446,  0.80064077],
       [-0.2376931 , -0.06235213, -0.77998182,  0.57553676],
       [-0.13007256,  0.98277509, -0.10089789, -0.08398726],
       [-0.90775493, -0.06800222,  0.38817359,  0.14379826]])

In [11]:
SP = np.zeros((S.shape[1], S.shape[0]))
for i in range(len(eigenval)):
    if eigenval[i] > 0:
        SP[i][i] = 1 / np.sqrt(eigenval[i])

SP

array([[0.05063697, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [28]:
MPInv = np.matmul(eigenvec.transpose(), SP)
MPInv = np.matmul(MPInv, UT)
MPInv = np.abs(MPInv)
MPInv

array([[0.00512821, 0.0025641 , 0.00769231, 0.01282051],
       [0.01538462, 0.00769231, 0.02307692, 0.03846154]])

In [32]:
np.matmul(np.matmul( MPInv, mat), MPInv)

array([[0.00512821, 0.0025641 , 0.00769231, 0.01282051],
       [0.01538462, 0.00769231, 0.02307692, 0.03846154]])